In [1]:
#Dependencies
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

In [4]:
#Dataset
data = fetch_movielens(min_rating=4.0)

In [6]:
#repr() is used to get string representation of the object
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [32]:
#Model is content based + collaborative
model1 = LightFM(loss='bpr')  #Bayesian Personalised Ranking pairwise loss
model2 = LightFM(loss='warp') #Weighted Approximate-Rank Pairwise loss

In [33]:
model1.fit(data['train'], epochs=30, num_threads=2)
model2.fit(data['train'], epochs=30, num_threads=2)

In [34]:
print(repr(data['item_labels']))

array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)


In [35]:
def sample_recommendations(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        #Rating 5 is positive ratings that are 4 or less are negative
        known_positive = data['item_labels'][data['train'].tocsr()[user_id].indices]
        scores=model.predict(user_id, np.arange(n_items))
        top_items=data['item_labels'][np.argsort(-scores)]
        print("User %s" % user_id)
        print("  Known positive: ")
        for x in known_positive[:3]:
            print("   %s" % x)
        print(" Recommendations:")
        for x in top_items[:3]:
            print("   %s" % x)    

In [36]:
sample_recommendations(model1, data, [3, 255, 420])

User 3
  Known positive: 
   Seven (Se7en) (1995)
   Contact (1997)
   Starship Troopers (1997)
 Recommendations:
   Air Force One (1997)
   L.A. Confidential (1997)
   Contact (1997)
User 255
  Known positive: 
   Toy Story (1995)
   GoldenEye (1995)
   Get Shorty (1995)
 Recommendations:
   Independence Day (ID4) (1996)
   Twister (1996)
   Rock, The (1996)
User 420
  Known positive: 
   Usual Suspects, The (1995)
   Star Wars (1977)
   Pulp Fiction (1994)
 Recommendations:
   Alien (1979)
   GoodFellas (1990)
   Terminator, The (1984)


In [37]:
sample_recommendations(model2, data, [3, 255, 420])

User 3
  Known positive: 
   Seven (Se7en) (1995)
   Contact (1997)
   Starship Troopers (1997)
 Recommendations:
   Contact (1997)
   Kiss the Girls (1997)
   Scream (1996)
User 255
  Known positive: 
   Toy Story (1995)
   GoldenEye (1995)
   Get Shorty (1995)
 Recommendations:
   Twister (1996)
   Independence Day (ID4) (1996)
   Jerry Maguire (1996)
User 420
  Known positive: 
   Usual Suspects, The (1995)
   Star Wars (1977)
   Pulp Fiction (1994)
 Recommendations:
   Star Wars (1977)
   Godfather, The (1972)
   Raiders of the Lost Ark (1981)
